# RadialGridGenerator Examples

The `RadialGridGenerator` class in the `power_grid_model_ds` library is designed to generate radial grid structures with nodes, sources, lines, and optionally transformers. This documentation covers the usage of the `RadialGridGenerator` class, including grid generation, node, source, and line generation, and various grid configuration functions.

## Grid Generation

### Generating a Random Grid

The `RadialGridGenerator` class can be used to generate a random grid with a specified structure. The `run` method initializes the grid generation process.


In [1]:
from power_grid_model_ds import Grid
from power_grid_model_ds.generators import RadialGridGenerator

# Generate a random grid
grid_generator = RadialGridGenerator(grid_class=Grid)
grid = grid_generator.run(seed=0)

# Verify the structure of the generated grid
print(f"Number of nodes: {len(grid.node)}")
print(f"Number of sources: {len(grid.source)}")
print(f"Number of loads: {len(grid.sym_load)}")

Number of nodes: 102
Number of sources: 2
Number of loads: 100


### Generating a Random Grid with Transformers

The RadialGridGenerator can also generate a grid that includes transformers.


In [2]:
from power_grid_model_ds import Grid
from power_grid_model_ds.generators import RadialGridGenerator

# Generate a random grid with transformers
grid_generator = RadialGridGenerator(grid_class=Grid)
grid = grid_generator.run(seed=0, create_10_3_kv_net=True)

# Verify the transformers in the generated grid
print(f"Number of transformers: {len(grid.transformer)}")

Number of transformers: 2


## Node Generation

The NodeGenerator class is used to generate random nodes within the grid.


In [3]:
from power_grid_model_ds import Grid
from power_grid_model_ds.arrays import NodeArray
from power_grid_model_ds.generators import NodeGenerator

# Generate random nodes
grid = Grid.empty()
node_generator = NodeGenerator(grid, seed=0)
nodes, loads_low, loads_high = node_generator.run(amount=2)

# Verify the generated nodes and loads
print(f"Generated nodes: \n{nodes}")
print(f"Low load scenarios: \n{loads_low}")
print(f"High load scenarios: \n{loads_high}")

Generated nodes: 
 id | u_rated | node_type | feeder_branch_id | feeder_node_id 
 1  | 10500.0 |     0     |        0         |       0        
 2  | 10500.0 |     0     |        0         |       0        
Low load scenarios: 
 id | node | status | type | p_specified | q_specified 
 3  |  1   |   1    |  0   |   218860.0  |   29606.0   
 4  |  2   |   1    |  0   |   180184.0  |   21574.0   
High load scenarios: 
 id | node | status | type | p_specified | q_specified 
 5  |  1   |   1    |  0   |  -287484.0  |   40640.0   
 6  |  2   |   1    |  0   |   26558.0   |   28148.0   


## Source Generation

The SourceGenerator class is used to generate random sources within the grid.


In [4]:
from power_grid_model_ds import Grid
from power_grid_model_ds.arrays import SourceArray
from power_grid_model_ds.generators import SourceGenerator

# Generate random sources
grid = Grid.empty()
source_generator = SourceGenerator(grid=grid, seed=0)
nodes, sources = source_generator.run(amount=1)

# Verify the generated nodes and sources
print(f"Generated nodes: \n{nodes}")
print(f"Generated sources: \n{sources}")

Generated nodes: 
 id | u_rated | node_type | feeder_branch_id | feeder_node_id 
 1  | 10500.0 |     1     |   -2147483648    |  -2147483648   
Generated sources: 
 id | node | status | u_ref 
 2  |  1   |   1    |  1.0  


## Line Generation

The LineGenerator class is used to generate random lines within the grid.


In [5]:
from power_grid_model_ds import Grid
from power_grid_model_ds.generators import LineGenerator

# Setup initial nodes and sources
grid = Grid.empty()
nodes = NodeArray.zeros(4)
nodes.id = [0, 1, 2, 3]
nodes.u_rated = [10_500] * 4

sources = SourceArray.zeros(1)
sources.id = [4]
sources.node = [0]
sources.status = [1]
sources.u_ref = [1]

grid.append(nodes)
grid.append(sources)

# Generate random lines
line_generator = LineGenerator(grid=grid, seed=0)
lines = line_generator.run(amount=2)

# Verify the generated lines
print(f"Generated lines: \n{lines}")

Generated lines: 
 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder |   r1  |   x1  |  c1 | tan1 |   i_n   
 5  |     0     |    3    |      1      |     1     |        0         |       0        |   False   |0.003..|4.538..| 0.0 | 0.0  |303.919..
 6  |     0     |    2    |      1      |     1     |        0         |       0        |   False   |0.134..|0.015..| 0.0 | 0.0  |425.988..
 7  |     3     |    1    |      1      |     1     |        0         |       0        |   False   |1.211..|0.065..| 0.0 | 0.0  |663.357..
 8  |     0     |    2    |      1      |     0     |        0         |       0        |   False   |0.629..|0.006..| 0.0 | 0.0  |313.880..
 9  |     3     |    2    |      1      |     0     |        0         |       0        |   False   |0.070..|0.029..| 0.0 | 0.0  |269.786..


## Grid Configuration Functions

### Creating Routes

The create_routes method in the LineGenerator class generates routes between nodes.


In [6]:
# Create routes
line_generator.create_routes(2)

# Verify the generated routes
print(f"Generated routes: \n{line_generator.line_array}")

Generated routes: 
 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder |   r1  |   x1  |  c1 | tan1 |  i_n   
 5  |     0     |    3    |      1      |     1     |        0         |       0        |   False   |0.347..|0.008..| 0.0 | 0.0  |306.03..
 6  |     0     |    1    |      1      |     1     |        0         |       0        |   False   |0.082..|0.037..| 0.0 | 0.0  |254.90..


### Connecting Nodes

The connect_nodes method connects unconnected nodes in the grid.


In [7]:
# Connect unconnected nodes
line_generator.set_unconnected_nodes()
line_generator.connect_nodes()

# Verify the connected nodes
print(f"Connected nodes: {line_generator.connected_nodes}")
print(f"Unconnected nodes: {line_generator.unconnected_nodes}")

Connected nodes: [0, 1, 3]
Unconnected nodes: [2]


### Creating Normally Open Points

The create_nop_lines method generates normally open points in the grid.


In [8]:
# Create normally open points
line_generator.create_nop_lines(1)

# Verify the generated normally open points
print(f"Generated NOP lines: \n{line_generator.line_array}")

Generated NOP lines: 
 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder |   r1  |   x1  |  c1 | tan1 |   i_n   
 5  |     0     |    3    |      1      |     1     |        0         |       0        |   False   |0.347..|0.008..| 0.0 | 0.0  |306.039..
 6  |     0     |    1    |      1      |     1     |        0         |       0        |   False   |0.082..|0.037..| 0.0 | 0.0  |254.909..
 7  |     0     |    2    |      1      |     1     |        0         |       0        |   False   |0.064..|0.004..| 0.0 | 0.0  |445.042..
 8  |     1     |    2    |      1      |     0     |        0         |       0        |   False   |0.209..|0.024..| 0.0 | 0.0  |270.937..
